In [63]:
import glob
import numpy as np
import pandas as pd 
import pickle
import json
import re
import nltk
import matplotlib.pyplot as plt
import random
import time
import multiprocessing as mp

from shutil import copyfile

from fuzzywuzzy import fuzz

from fuzzywuzzy import process

import csv

In [64]:
import os

In [65]:
# Trim the potential titles
titles = pd.read_csv("../data/title_item_jujun2.csv")
titles['num_words'] = titles.Title.apply(lambda x: len(re.findall("[a-zA-Z_]+",x)))
titles.tail()

,Title,s,item_number,num_words
83,item management discussion and analysis of fin...,33,item7,12
84,item exhibits and financial statements schedules,31,item15,6
85,item selected financial data in thousands exce...,30,item6,10
86,item directors and officers of the registrant,29,item10,7
87,item submission of matters to vote of security...,9999,item4,9


In [66]:
def get_text(f):
    path = '/backup_disk/backup/pyproject/fin/data/edgar_text/'
    #path = '/home/jujun/temp_10k/data_1014/test_files/'
    #filename = '-'.join(f.split('/')[-2:])
    path_file = path + f
    
    
    with open(path_file,'rb') as fread:
        text = fread.read()
        text = text.decode('utf8').encode("ascii", "ignore")
        text = text.decode('utf8')
        text = text.lower()
        #print(text[0:100])
    fread.close()
    return text

In [67]:
# get disclosure outline
def get_outline(doc, thr):
    
    text = doc#.decode('utf8')
    outline = [] 
    ps = []
    for match in re.finditer(r'(?s)((?:[^\n][\n]?)+)', text):
        s = match.start()
        e = match.end()
        p = match.group()
        
        p1 = re.findall("[a-zA-Z]{2,}", p.lower())        
        
        if len(p1)>=2 and len(p1)<25:
            ps.append([' '.join(p1), s, e])       
    
    # This may need to speed up using multiple threading
    for idx, row in titles.iterrows():
        #print(row["Title"],row["item_number"] )
        for item in ps:
            n = len(re.findall("[a-zA-Z_]+", item[0]))
            if n-2< row['num_words'] <n+2:
                score = fuzz.ratio(item[0], row["Title"])
                score_p = fuzz.partial_ratio(item[0], row["Title"])
                if score > thr:
                    outline.append([row["Title"],row["item_number"] , score]+item)
                elif score_p > 93:
                    outline.append([row["Title"],row["item_number"] , score_p]+item)
                else:pass
            else:
                pass
    outline = pd.DataFrame(outline, columns=["Title","item_number","Score","Text", "Start","End",]) 
    outline = outline.drop_duplicates(subset = ['Start','End'],keep ='last')
    outline = outline.sort_values(by="Start").reset_index(drop = True)
    return outline

In [68]:
def clean_text(text):
    paras_cl = []
    paras = text.split("\n\n")
    #print(paras)
    for para in paras:
        #print(para)
        p1 = re.findall("[a-zA-Z]{1,}", para.lower())
        num_p = len(p1)
        #print(num_p)

        p2 = re.sub("[^a-zA-Z]"," ",para)
        space_num = p2.count(' ')
        #print(space_num)

        if num_p*2 < space_num:
            para = ""   
        elif num_p <= 25:
            para = ""
        else:pass
        paras_cl.append(para)
        
    para_cl_fl = list(filter(None,paras_cl))
    para_cl_fl_2 = [re.sub(r"\n"," ",i) for i in para_cl_fl]
    
    return para_cl_fl_2

In [69]:
def imfor_structure(text,outline):
    r_dict = {}
    r_2_dict = {}
    outline_index = outline.index.tolist()
    num_max = len(text)

    for idxm,item_num in outline.item_number.iteritems():
        if item_num not in r_dict.keys():
            if idxm+1 in outline_index:
                r_dict[item_num] = [(outline.loc[idxm,'Start'],outline.loc[idxm+1,'Start'])]
            else: r_dict[item_num] = [(outline.loc[idxm,'Start'],num_max)]
        else: 
            if idxm+1 in outline_index:
                r_dict[item_num].append((outline.loc[idxm,'Start'],outline.loc[idxm+1,'Start']))
            else: r_dict[item_num] = [(outline.loc[idxm,'Start'],num_max)]

    for k,vs in r_dict.items():
        num_para = []
        for v in vs:
            text_temp = text[v[0]:v[1]]
            text_temp_cl = clean_text(text_temp)
            num_para.append(len(text_temp_cl))
        r_2_dict[k] = sum(num_para)
    return r_2_dict

In [70]:
def get_item(doc,outline,item_name):
    text = [] 

    pattern = r"\b" + item_name + r"[a-z]?\b"
    selected = titles[titles.item_number.str.match(pat = pattern)]
    mask = outline.Title.apply(lambda x: any(item for item in selected.Title if item == x))
    target = outline[mask]
   

    target = target.drop_duplicates(subset =['Start','End'],keep = 'last')

    if len(target) > 0: 
        outline_index = outline.index.tolist()
        target_index = target.index.tolist()
        #print(outline_index)
        for idx in target_index:
            #print(type(idx))
            #print(idx+1 in outline_index)
            if idx+1 in outline_index:
                
                txt_ps = doc[int(outline.loc[idx,'Start']):int(outline.loc[idx+1,'Start'])]

            else:
                
                txt_ps = doc[int(outline.loc[idx,'End']):len(doc)]

            text.append(txt_ps)                
        
        
    return '\n\n'.join(text)

In [71]:
def save_file(f,text_fl,item_name):
    save_path = '/home/jujun/pyproject/research/jujun/frauds_files/'
    #save_path = '/home/jujun/temp_10k/data_1014/test_save/'
    filename = '-'.join(f.split('/')[-2:])
    save_file_path = save_path + item_name + '/' + filename
    #print(save_file_path)
    with open(save_file_path, "w") as output_file:
        output_file.write(text_fl)
        
#         writer = csv.writer(output_file, delimiter = ",")
#         for t_ in text_fl:
#             writer.writerow(t_)
        #output_file.write(json.dumps(text_fl))
    output_file.close()     
    return save_file_path

In [72]:
def get_num_para_words(text_fl):
    temp_cl = clean_text(text_fl)
    num_para = len(temp_cl)
    
    temp_cl_2 = " ".join(temp_cl)
    num_words = len(re.findall("[a-zA-Z]{1,}", temp_cl_2))
    return num_para,num_words

In [73]:
def sub_main(f,text,outline,item_name):
    text_fl = get_item(text,outline,item_name)
    #print(text_fl[0:100])
    save_file_path = save_file(f,text_fl,item_name)
    #num_para,num_words = get_num_para_words(text_fl)
    info = [save_file_path]
    return info 

In [74]:
def main(f,thr):
    try:
        text = get_text(f)
        outline = get_outline(text,thr)
        #print(outline)
        summary_dict = imfor_structure(text,outline)
        info_7 = sub_main(f,text,outline,'item7')
#         info_2 = sub_main(f,text,outline,'item2')
#         info_1 = sub_main(f,text,outline,'item1')

    except:
        info_7 = ["fail"]
#         info_2 = ["fail","fail","fail","fail"]
#         info_1 = ["fail","fail","fail","fail"]
    
    return [f,info_7,summary_dict]

In [75]:
fraud_record = pd.read_excel('../data/DGLS_20160930_D.xlsx',sheet_name = 'ann')

In [76]:
fraud_record

,ID,CONAME,CIK,YEARA,FYR,DATE,P_AAER,UNDERSTATEMENT,YEAR
0,2,"21st Century Technologies, Inc.",1090870.0,2003.0,12.0,2003-12-31,2869,0.0,2003.0
1,4,"3CI, Incorporated, now known as Cordero Indust...",718388.0,1984.0,3.0,1985-03-31,271,0.0,1985.0
2,4,"3CI, Incorporated, now known as Cordero Indust...",718388.0,1985.0,3.0,1986-03-31,271,0.0,1986.0
3,4,"3CI, Incorporated, now known as Cordero Indust...",718388.0,1986.0,3.0,1987-03-31,271,0.0,1987.0
4,5,"3Net Systems, Inc. now known as ALTERNATIVE TE...",885520.0,1991.0,6.0,1991-06-30,833,0.0,1991.0
5,6,"800 America.com, Inc.",1060239.0,2001.0,12.0,2001-12-31,1662,0.0,2001.0
6,8,A.C.L.N. Ltd.,1021264.0,1998.0,12.0,1998-12-31,2105,0.0,1998.0
7,8,A.C.L.N. Ltd.,1021264.0,1999.0,12.0,1999-12-31,2105,0.0,1999.0
8,8,A.C.L.N. Ltd.,1021264.0,2000.0,12.0,2000-12-31,2105,0.0,2000.0
9,10,"A.T. Bliss and Company, Inc. (now OMNI Equitie...",351038.0,1979.0,12.0,1979-12-31,20,0.0,1979.0


In [77]:
fraud_cik_list = list(set(fraud_record.CIK.dropna()))

In [78]:
fraud_cik_list = [int(l) for l in fraud_cik_list]

In [79]:
fraud_cik_list

[1122304,
 882692,
 4107,
 923662,
 1343504,
 917524,
 313368,
 1089567,
 356395,
 49196,
 890923,
 763950,
 897074,
 88121,
 319546,
 30780,
 718909,
 18498,
 313413,
 1091667,
 933974,
 2135,
 888919,
 1108058,
 354396,
 890981,
 1118326,
 893046,
 352383,
 1106056,
 6284,
 1312910,
 84112,
 1095827,
 929940,
 811156,
 923796,
 28823,
 71829,
 1198233,
 26780,
 73887,
 84129,
 1026214,
 727207,
 714921,
 731306,
 1058986,
 731308,
 41133,
 225457,
 794803,
 215219,
 1007800,
 704700,
 18621,
 919742,
 1280191,
 1065152,
 28864,
 913600,
 803014,
 731336,
 929994,
 884940,
 1009869,
 1407185,
 1331427,
 927971,
 108772,
 866535,
 90350,
 352495,
 80124,
 798985,
 1050891,
 932112,
 721171,
 819479,
 837912,
 737561,
 12570,
 1399067,
 813342,
 737572,
 700711,
 1382696,
 51497,
 891178,
 784683,
 706860,
 813359,
 1057083,
 1081661,
 1030471,
 1085776,
 1024338,
 1065298,
 878932,
 1210708,
 764244,
 72020,
 1087831,
 876889,
 1603923,
 911707,
 876896,
 913760,
 860518,
 311657,
 135

In [80]:
output = []
error = []
for c in fraud_cik_list:
    try:
        list_files = os.listdir("/backup_disk/backup/pyproject/fin/data/edgar_text/edgar/data/" + str(c)+'/')
        for f in list_files:
            file = 'edgar/data/' + str(c) + '/' + f
            r = main(file,85)
            output.append(r)
    except:
        error.append(c)

In [81]:
result_tabel = pd.DataFrame(output,columns=["file","information_item7","structure"])

In [82]:
result_tabel.shape

(6364, 3)

### Text Extract Done

In [83]:
result_tabel_1 = result_tabel.copy()

In [84]:
info_reg_stack = {"comp_name": r'\s+company conformed name:\s+(.{3,100})',
             "SIC": r'\s+standard industrial classification:\s+(.{3,100})',
             'CIK': r'\s+central index key:\s+(\d{10})',
             "date": r'\s+conformed period of report:\s+(\d{8})',
             "type": r'\s+conformed submission type:\s+(.{1,20})'   #"type": r'\s+conformed submission type:\s+[10](.{3,20})'    
                 }

pattern = '(\s*item\s+\d+[a-zA-Z]?[\W_]*\w+)'

In [85]:
def getInfo(filepath,info_reg_stack,pattern):

    result_dic = {}
    
    with open(filepath) as f:
        content = re.split(pattern, f.read().lower().replace('*',''))

    for key, item in info_reg_stack.items():
        if re.search(item,content[0]):
            result_dic[key]=re.search(item,content[0]).group(1)
    
    return result_dic

In [86]:
result_tabel_1['SIC'] = 99
result_tabel_1['CIK'] = 99
result_tabel_1['date'] = 99
result_tabel_1['type'] = 99
result_tabel_1['comp_name'] = 99

In [87]:
path = '/backup_disk/backup/pyproject/fin/data/edgar_text/'
error = []
for i, f in result_tabel_1.file.iteritems():
    if i%100 == 0:
        print(i)
        
    path_file = path + f 
    result = getInfo(path_file,info_reg_stack,pattern)
    try:
        result_tabel_1.iloc[i,3] = result['SIC']
        result_tabel_1.iloc[i,4] = result['CIK']
        result_tabel_1.iloc[i,5] = result['date']
        result_tabel_1.iloc[i,6] = result['type']
        result_tabel_1.iloc[i,7] = result['comp_name']

    except:
        error.append(f)



0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300


In [88]:
result_tabel_1

,file,information_item7,structure,SIC,CIK,date,type,comp_name
0,edgar/data/1122304/0001122304-16-000201.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,20151231,10-k,aetna inc /pa/
1,edgar/data/1122304/0000950123-03-002268.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 4, 'item6': 0, 'item7a': 0, 'item3'...",hospital & medical service plans [6324],0001122304,20021231,10-k,aetna inc /pa/
2,edgar/data/1122304/0000950123-04-002526.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 4, 'item15': 0, 'item6': 0, 'item7a...",hospital & medical service plans [6324],0001122304,20031231,10-k,aetna inc /pa/
3,edgar/data/1122304/0001122304-15-000038.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,20141231,10-k,aetna inc /pa/
4,edgar/data/1122304/0001122304-12-000045.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,20111231,10-k,aetna inc /pa/
5,edgar/data/1122304/0000950123-07-002780.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 4, 'item6': 0, 'item7a': 1, 'item3'...",hospital & medical service plans [6324],0001122304,20061231,10-k,aetna inc /pa/
6,edgar/data/1122304/0001122304-08-000034.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 4, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,20071231,10-k,aetna inc /pa/
7,edgar/data/1122304/0000950123-06-002414.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item7a': 0, 'item3'...",hospital & medical service plans [6324],0001122304,20051231,10-k,aetna inc /pa/
8,edgar/data/1122304/0001122304-10-000024.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,20091231,10-k,aetna inc /pa/
9,edgar/data/1122304/0001122304-11-000032.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,20101231,10-k,aetna inc /pa/


In [129]:
def extract_r(x):
    r = 0
    try:
        r = x['item7'] + x['item7a']
    except:
        r = 0        
    return r

In [130]:
 result_tabel_1['success'] = result_tabel_1.structure.apply(lambda x: extract_r(x))

In [131]:
result_tabel_1

,file,information_item7,structure,SIC,CIK,date,type,comp_name,success
0,edgar/data/1122304/0001122304-16-000201.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,20151231,10-k,aetna inc /pa/,1
1,edgar/data/1122304/0000950123-03-002268.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 4, 'item6': 0, 'item7a': 0, 'item3'...",hospital & medical service plans [6324],0001122304,20021231,10-k,aetna inc /pa/,0
2,edgar/data/1122304/0000950123-04-002526.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 4, 'item15': 0, 'item6': 0, 'item7a...",hospital & medical service plans [6324],0001122304,20031231,10-k,aetna inc /pa/,1
3,edgar/data/1122304/0001122304-15-000038.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,20141231,10-k,aetna inc /pa/,1
4,edgar/data/1122304/0001122304-12-000045.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,20111231,10-k,aetna inc /pa/,1
5,edgar/data/1122304/0000950123-07-002780.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 4, 'item6': 0, 'item7a': 1, 'item3'...",hospital & medical service plans [6324],0001122304,20061231,10-k,aetna inc /pa/,1
6,edgar/data/1122304/0001122304-08-000034.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 4, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,20071231,10-k,aetna inc /pa/,1
7,edgar/data/1122304/0000950123-06-002414.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item7a': 0, 'item3'...",hospital & medical service plans [6324],0001122304,20051231,10-k,aetna inc /pa/,0
8,edgar/data/1122304/0001122304-10-000024.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,20091231,10-k,aetna inc /pa/,1
9,edgar/data/1122304/0001122304-11-000032.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,20101231,10-k,aetna inc /pa/,1


In [132]:
result_tabel_1.success.value_counts()

0      2071
2       209
1       130
67       39
46       37
34       36
89       34
73       34
92       33
76       32
66       32
62       32
54       32
41       32
91       32
60       32
48       32
94       31
86       31
79       31
42       31
64       31
49       31
53       31
38       30
71       30
74       30
95       30
59       30
68       29
       ... 
387       1
395       1
720       1
692       1
672       1
656       1
257       1
277       1
301       1
313       1
685       1
589       1
585       1
569       1
557       1
545       1
541       1
537       1
529       1
517       1
461       1
437       1
433       1
425       1
421       1
393       1
349       1
329       1
321       1
763       1
Name: success, Length: 428, dtype: int64

In [136]:
result_tabel_2 = result_tabel_1[result_tabel_1.date != 99]

In [137]:
from datetime import datetime

In [138]:
result_tabel_2['date'] = result_tabel_2.date.apply(lambda x: str(x))
result_tabel_2['date'] = result_tabel_2.date.apply(lambda x: datetime.strptime(x, '%Y%m%d'))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [139]:
def get_yeara(x):
    year = x.year
    if x.month <5:
        year -= 1        
    return year

In [140]:
result_tabel_2['yeara'] = result_tabel_2.date.apply(lambda x: get_yeara(x))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [141]:
result_tabel_2['map_key'] = result_tabel_2.CIK.apply(lambda x: str(int(x))) + "_" + result_tabel_2.yeara.apply(lambda x: str(int(x)))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [142]:
result_tabel_2

,file,information_item7,structure,SIC,CIK,date,type,comp_name,success,yeara,map_key
0,edgar/data/1122304/0001122304-16-000201.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,2015-12-31,10-k,aetna inc /pa/,1,2015,1122304_2015
1,edgar/data/1122304/0000950123-03-002268.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 4, 'item6': 0, 'item7a': 0, 'item3'...",hospital & medical service plans [6324],0001122304,2002-12-31,10-k,aetna inc /pa/,0,2002,1122304_2002
2,edgar/data/1122304/0000950123-04-002526.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 4, 'item15': 0, 'item6': 0, 'item7a...",hospital & medical service plans [6324],0001122304,2003-12-31,10-k,aetna inc /pa/,1,2003,1122304_2003
3,edgar/data/1122304/0001122304-15-000038.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,2014-12-31,10-k,aetna inc /pa/,1,2014,1122304_2014
4,edgar/data/1122304/0001122304-12-000045.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,2011-12-31,10-k,aetna inc /pa/,1,2011,1122304_2011
5,edgar/data/1122304/0000950123-07-002780.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 4, 'item6': 0, 'item7a': 1, 'item3'...",hospital & medical service plans [6324],0001122304,2006-12-31,10-k,aetna inc /pa/,1,2006,1122304_2006
6,edgar/data/1122304/0001122304-08-000034.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 4, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,2007-12-31,10-k,aetna inc /pa/,1,2007,1122304_2007
7,edgar/data/1122304/0000950123-06-002414.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item7a': 0, 'item3'...",hospital & medical service plans [6324],0001122304,2005-12-31,10-k,aetna inc /pa/,0,2005,1122304_2005
8,edgar/data/1122304/0001122304-10-000024.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,2009-12-31,10-k,aetna inc /pa/,1,2009,1122304_2009
9,edgar/data/1122304/0001122304-11-000032.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,2010-12-31,10-k,aetna inc /pa/,1,2010,1122304_2010


In [143]:
fraud_table = fraud_record.dropna()

In [144]:
fraud_table['map_key'] = fraud_table.CIK.apply(lambda x: str(int(x))) + "_" + fraud_table.YEARA.apply(lambda x: str(int(x)))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [145]:
def match(x,list_file):
    m=0
    if x in list_file:
        m=1 
    return m

In [146]:
fraud_table_list = fraud_table.map_key.tolist()

In [147]:
result_tabel_2['fraud_match'] = result_tabel_2.map_key.apply(lambda x: match(x,fraud_table_list))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [148]:
result_tabel_2

,file,information_item7,structure,SIC,CIK,date,type,comp_name,success,yeara,map_key,fraud_match
0,edgar/data/1122304/0001122304-16-000201.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,2015-12-31,10-k,aetna inc /pa/,1,2015,1122304_2015,0
1,edgar/data/1122304/0000950123-03-002268.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 4, 'item6': 0, 'item7a': 0, 'item3'...",hospital & medical service plans [6324],0001122304,2002-12-31,10-k,aetna inc /pa/,0,2002,1122304_2002,0
2,edgar/data/1122304/0000950123-04-002526.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 4, 'item15': 0, 'item6': 0, 'item7a...",hospital & medical service plans [6324],0001122304,2003-12-31,10-k,aetna inc /pa/,1,2003,1122304_2003,0
3,edgar/data/1122304/0001122304-15-000038.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,2014-12-31,10-k,aetna inc /pa/,1,2014,1122304_2014,0
4,edgar/data/1122304/0001122304-12-000045.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,2011-12-31,10-k,aetna inc /pa/,1,2011,1122304_2011,0
5,edgar/data/1122304/0000950123-07-002780.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 4, 'item6': 0, 'item7a': 1, 'item3'...",hospital & medical service plans [6324],0001122304,2006-12-31,10-k,aetna inc /pa/,1,2006,1122304_2006,0
6,edgar/data/1122304/0001122304-08-000034.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 4, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,2007-12-31,10-k,aetna inc /pa/,1,2007,1122304_2007,0
7,edgar/data/1122304/0000950123-06-002414.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item7a': 0, 'item3'...",hospital & medical service plans [6324],0001122304,2005-12-31,10-k,aetna inc /pa/,0,2005,1122304_2005,0
8,edgar/data/1122304/0001122304-10-000024.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,2009-12-31,10-k,aetna inc /pa/,1,2009,1122304_2009,0
9,edgar/data/1122304/0001122304-11-000032.txt,[/home/jujun/pyproject/research/jujun/frauds_f...,"{'item12': 1, 'item6': 0, 'item9b': 0, 'item3'...",hospital & medical service plans [6324],0001122304,2010-12-31,10-k,aetna inc /pa/,1,2010,1122304_2010,0


In [165]:
result_tabel_2.shape

(6273, 12)

In [167]:
result_tabel_3 = result_tabel_2.copy()

In [168]:
result_tabel_3.shape

(6273, 12)

In [169]:
result_tabel_3['fraud_match_lag'] = 0

In [170]:
error_list = []
nofound_list = []
c = 0
for i in result_tabel_3.index:
    c += 1
    if c%1000 == 0:
        print(c)
        
    if result_tabel_3.fraud_match[i] == 1:
        look_map = str(int(result_tabel_3.CIK[i])) +'_'+ str(result_tabel_3.yeara[i]-1)
        if len(result_tabel_3[result_tabel_3.map_key == look_map]) >= 1:
            target = result_tabel_3[result_tabel_3.map_key == look_map]
            result_tabel_3.fraud_match_lag[target.index] = 1
#         elif len(result_tabel_3[result_tabel_3.map_key == look_map]) > 1:
#             print('error infor_tb.index',i)
#             error_list.append(result_tabel_3.map_key[i])
        elif len(result_tabel_3[result_tabel_3.map_key == look_map]) == 0:
            print('nofound infor_tb.index',i)
            nofound_list.append(result_tabel_3.map_key[i])
        else: print('other infor_tb.index',i)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


nofound infor_tb.index 39
nofound infor_tb.index 115
nofound infor_tb.index 214
nofound infor_tb.index 326
nofound infor_tb.index 442
nofound infor_tb.index 580
nofound infor_tb.index 621
nofound infor_tb.index 673
nofound infor_tb.index 674
nofound infor_tb.index 860
nofound infor_tb.index 913
nofound infor_tb.index 961
nofound infor_tb.index 989
1000
nofound infor_tb.index 1042
nofound infor_tb.index 1206
nofound infor_tb.index 1395
nofound infor_tb.index 1427
nofound infor_tb.index 1434
nofound infor_tb.index 1436
nofound infor_tb.index 1475
nofound infor_tb.index 1586
nofound infor_tb.index 1705
nofound infor_tb.index 1734
nofound infor_tb.index 1737
nofound infor_tb.index 1806
nofound infor_tb.index 1836
nofound infor_tb.index 1857
nofound infor_tb.index 1881
nofound infor_tb.index 1928
nofound infor_tb.index 1943
nofound infor_tb.index 1971
nofound infor_tb.index 2025
2000
nofound infor_tb.index 2111
nofound infor_tb.index 2275
nofound infor_tb.index 2344
nofound infor_tb.index 2

In [173]:
sum(result_tabel_3.fraud_match)

1011

In [174]:
sum(result_tabel_3.fraud_match_lag)

929

In [193]:
result_tabel_3.CIK = result_tabel_3.CIK.apply(lambda x: int(x))

In [195]:
files_count = dict()
for c in  fraud_cik_list: 
    v = []
    try:
        list_files = os.listdir("/backup_disk/backup/pyproject/fin/data/edgar_text/edgar/data/" + str(c)+'/')
        v.append(len(list_files))
        temp = result_tabel_3[(result_tabel_3.CIK == c)& (result_tabel_3.success > 0)]
        v.append(len(temp))
        files_count[c] = v
    except:
        v.append(0)
        temp = result_tabel_3[(result_tabel_3.CIK == c)& (result_tabel_3.success > 0)]
        v.append(len(temp))
        files_count[c] = v
        


In [196]:
files_count

{2135: [14, 9],
 2463: [0, 0],
 2491: [20, 17],
 3116: [23, 19],
 3449: [3, 0],
 3662: [4, 0],
 3905: [0, 0],
 3906: [15, 3],
 4107: [0, 0],
 4962: [24, 20],
 5272: [24, 15],
 6284: [24, 17],
 8868: [24, 18],
 12152: [0, 0],
 12570: [15, 11],
 13055: [8, 1],
 14272: [24, 15],
 16926: [0, 0],
 18498: [24, 19],
 18621: [0, 0],
 21344: [24, 16],
 21759: [10, 4],
 23026: [0, 0],
 23082: [24, 19],
 23217: [24, 14],
 25095: [13, 4],
 26172: [24, 15],
 26780: [24, 19],
 27748: [16, 13],
 27960: [13, 4],
 28823: [23, 19],
 28864: [0, 0],
 29534: [23, 18],
 29693: [8, 2],
 29850: [4, 1],
 29915: [23, 19],
 30780: [0, 0],
 32120: [4, 0],
 32160: [0, 0],
 35214: [24, 16],
 35527: [24, 20],
 36161: [2, 0],
 36570: [0, 0],
 40545: [24, 9],
 40562: [0, 0],
 40570: [23, 11],
 41133: [17, 8],
 42284: [16, 5],
 45012: [24, 13],
 46431: [0, 0],
 47059: [0, 0],
 49196: [23, 17],
 49534: [16, 7],
 49728: [22, 18],
 50441: [0, 0],
 50493: [24, 20],
 51124: [8, 5],
 51143: [24, 1],
 51497: [0, 0],
 51644: [

In [202]:
files_count_tb = pd.DataFrame.from_dict(files_count, orient='index',columns=['local_docs', 'obtain_docs']).reset_index()

In [204]:
files_count_tb.head()

,index,local_docs,obtain_docs
0,1122304,17,12
1,882692,4,0
2,4107,0,0
3,923662,0,0
4,1343504,4,0


In [206]:
files_count_tb.shape

(688, 3)

In [210]:
sum(files_count_tb.iloc[:,2])/sum(files_count_tb.iloc[:,1])

0.6623192960402263

In [205]:
files_count_tb_1 = files_count_tb[(files_count_tb.local_docs != 0) & (files_count_tb.obtain_docs != 0)]

In [207]:
files_count_tb_1.shape

(454, 3)

In [209]:
files_count_tb_1

,index,local_docs,obtain_docs
0,1122304,17,12
5,917524,10,7
7,1276591,10,10
8,1089567,4,4
10,356395,6,2
11,49196,23,17
12,763950,21,8
14,88121,24,19
17,718909,9,4
18,1067701,19,13


In [211]:
sum(files_count_tb_1.iloc[:,2])/sum(files_count_tb_1.iloc[:,1])

0.7068589636089216

In [212]:
result_tabel_3.shape

(6273, 13)

In [214]:
files_count_lg = dict()
for c in  fraud_cik_list: 
    v = []
    try:
        list_files = os.listdir("/backup_disk/backup/pyproject/fin/data/edgar_text/edgar/data/" + str(c)+'/')
        v.append(len(list_files))
        temp = result_tabel_3[result_tabel_3.CIK == c]
        v.append(len(temp))
        files_count_lg[c] = v
    except:
        v.append(0)
        temp = result_tabel_3[result_tabel_3.CIK == c]
        v.append(len(temp))
        files_count_lg[c] = v

In [215]:
files_count_tb_lg = pd.DataFrame.from_dict(files_count_lg, orient='index',columns=['local_docs', 'obtain_docs']).reset_index()

In [220]:
files_count_tb_lg = files_count_tb_lg[files_count_tb_lg.local_docs != 0]

In [221]:
sum(files_count_tb_lg.iloc[:,2])/sum(files_count_tb_lg.iloc[:,1])

0.978786926461345

In [219]:
result_tabel_3.to_csv('frauds_table_0520.csv')